## To Do: 
* standard data types
* clean data
** clean corrupted data points
** ensure uniform formatting
** handle missing values
* explore relationships among data (tbd)
* feature engineering
* do modeling

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

import statsmodels.api as sm

import sklearn.linear_model as lm
import sklearn.feature_selection as fs
from sklearn.metrics import r2_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
df = pd.read_csv('train.csv')
Y = df['SalePrice']
df = df.drop(['SalePrice'], axis=1)

In [ ]:
df.columns
df.head().loc[0]

## Data Exploration I

Upon doing a quick inspection of the data, I've devised with the following feature handling strategy: 
- variables with less than 20 unique values will be treated as categorical features, with the exception of
    - PoolArea
- the following features have more than 20 distinct values but will be treated as categorical
    - Neighborhood
    - YearBuilt
    - YearRemodAdd
    - GarageYrBlt
- the following features have less than 20 unique values, but will more appropriately be treated as continuous features
    - BsmtFullBath
    - BsmtHalfBath
    - FullBath
    - HalfBath
    - BedroomAbvGr
    - KitchenAbvGr
    - TotRmsAbvGrd
    - Fireplaces
    - GarageCars
    - MoSold

In [ ]:
not_converted = []
for col in df.columns:
    if (len(df[col].value_counts()) < 20):
        if col != 'PoolArea':
            df[col] = df[col].astype('category')
#             print(col + ' converted to Category')
    else:
        not_converted.append(col)
        
# Categorical variables that have more than 20 distinct values.
df['Neighborhood'] = df['Neighborhood'].astype('category')
df['YearBuilt'] = df['YearBuilt'].astype('category')
df['YearRemodAdd'] = df['YearRemodAdd'].astype('category')
df['GarageYrBlt'] = df['GarageYrBlt'].dropna().astype(int).astype('category')

# Variables with less than 20 distinct values, that are not categorical. They contain counts, and 
# will be treated as continuous/numeric.
to_convert = ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 
              'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 
              'Fireplaces', 'GarageCars']

for col in to_convert:
    # convert cols back to int. 
    df[col] = df[col].astype(int)

In [ ]:
# Separate out columns by data type to fill in missing values
integer_columns = df.select_dtypes(include=[np.int64, np.float64])
categorical_columns = df.select_dtypes(include=['category'])


In [ ]:
# plot the numerical variables
_ = integer_columns.hist(bins=10, figsize=(15, 20))


Observations: 
    - The variables that are approximately normally distributed or are normal with a skew, tend to be related to square footage.
    - There are a handful of discrete numerical variables representing counts
    - There are about 9 variables that look like they have a very low variability in information aka have a high percentage of one value.
    
    
    

In [ ]:
# plot the categorical variables

try:
    fig, ax = plt.subplots(11, 4)
    for i, cat in enumerate(categorical_columns.columns):
        _ = categorical_columns[cat].value_counts().plot('bar', ax=ax[i//4][i % 4], figsize=(15, 80)).set_title(cat);
    plt.show();
except IndexError:
    pass

## Data Cleaning


### High percentage of NAs

Identifying variables with more than some percentage of NAs

In [ ]:
thresh = .8
nulls = df.isna().sum()

fig = plt.figure(figsize=(8, 6))
plt.barh(nulls[nulls > df.shape[0]*thresh].sort_values(ascending=False).index, 
         (nulls[nulls > df.shape[0]*thresh].sort_values(ascending=False)/df.shape[0]*100).values)
fig.suptitle('Columns with percentage of missing values greater than 80%', fontsize=15)
plt.xlabel('%', fontsize=12)
plt.show();

This plot suggests we should drop Alley, PoolQC, Fence, MiscFeature because they are over 80% null.
Upon looking at the data dict however, you'll see that the NAs represent the case where a house does not have a given feature, and are not actually `nulls`. 
Let this be a reminder to understand what the data represents.

### Fill missing values

In [ ]:
# Interpolate two fields using linear interpolation
integer_columns = integer_columns.interpolate()

In [ ]:
# Handling the two fields where the NaNs do not correspond to a valid value.
# Fields 'Electrical' and "MasVnrType" have 8 and 1 missing values respectively. Since these are two fields
# where the NaN does not correspond to a value, and since there are so few records, we're dropping those.
categorical_columns = categorical_columns[~categorical_columns['Electrical'].isna()]
categorical_columns = categorical_columns[~categorical_columns['MasVnrType'].isna()]

# adding 'None' category to categorical fields, and then replacing NaN with 'None' for processing
def add_category(df, col):
    df[col].cat.add_categories('None', inplace=True)
null_categorical = categorical_columns.isna().sum().sort_values(ascending=False).reset_index()
for index, row in null_categorical.iterrows():
    categorical_columns[row['index']].cat.categories
    if row[0] > 0:
        add_category(categorical_columns, row['index'])
        categorical_columns[row['index']] = categorical_columns[row['index']].fillna(value='None')

### Handling categorical variables

Let's build a model with variables that have a covariance below a certain threshold. When running a covariance matrix on the dataframe, we find that the covariances range from -0.04 to 100,000. Let's set the covariance threshold at 5000 and see what kind of results we get.

In [ ]:
test = df.cov()[abs(df.cov()) < 5000]

test['LotFrontage'].loc[~test['LotFrontage'].isnull()].index

# for col in test.columns:
    

In [ ]:
# Identifying variables with a covariance higher than 10,000
high_cov_cols = dict()
gt_thresh = df.cov()[abs(df.cov()) > 10000]
for col in df.cov():
    high_cov_cols[col] = list(df.cov().index[gt_thresh[col].notnull()])
# 

In [ ]:
high_cov_cols

In [ ]:
X = df['LotArea']
Y = df['SalePrice']
# X = sm.add_constant(X)

In [ ]:
model = sm.OLS(endog=Y, exog=X.astype(float))
results = model.fit()
predictions = results.predict()

results.summary()

In [ ]:
# Fields to check correlations between
-

df.corr()[df.corr() > 0.4]

In [ ]:
# Separate out columns by data type to fill in missing values
integer_columns = df.select_dtypes(include=[np.int64, np.float64])
categorical_columns = df.select_dtypes(include=['category'])

# plt.hist(integer_columns, bins=10)
_ = integer_columns.hist(bins=10, figsize=(15, 20))


In [ ]:
categorical_columns.head()

In [ ]:
fig, ax = plt.subplots(11, 4)
for i, cat in enumerate(categorical_columns.columns):
    categorical_columns[cat].value_counts().plot('bar', ax=ax[i//4][i % 4], figsize=(15, 50)).set_title(cat);
    
plt.show();

In [ ]:
# df_filled : original df with missing values filled in
# rebuild df with integer columns and categorical columns, with NAs filled in.
# interpolated integer columns with the default settings, no limits on the nature of the interpolation
# doing a forward fill and backfill on categorical columns
df_filled = pd.concat([integer_columns.interpolate(), categorical_columns.ffill().bfill()], axis=1)

In [ ]:
X = pd.get_dummies(df_filled.drop(['SalePrice'], axis=1)).dropna()
y = df_filled['SalePrice']

In [ ]:
X.shape

In [ ]:
X.columns[~feat_sel.get_support()]

In [ ]:
# Experimenting with Feature Selection. Select features with a variance greater than 0.1.
feat_sel = fs.VarianceThreshold(threshold=(0.1))
var_sel = feat_sel.fit_transform(X)

# from X, drop the columns for the features whose variance did not meet the threshold, and 
# get dummy variables for remaining categorical features.
X2 = pd.get_dummies(X.drop(X.columns[~feat_sel.get_support()], axis=1)).dropna()

In [ ]:
# fit linear models on X (original with dummy variables for categorical) and X2 (with feature selection for V[x] > 0.1)
reg = lm.LinearRegression().fit(X, y)
reg

reg2 = lm.LinearRegression().fit(X2, y)
reg2

# Ridge Regression | L2 norm aka Euclidean norm aka the sqrt(sum(abs(Beta)^2)). Regularizes
# the loss with lambda*sum(Beta^2)
# NOTE: Increasing tolerance to 10, and max_iter to 10^6 did not bring the objective function to converge
reg3 = lm.Ridge(alpha=0.5, tol=10, max_iter=1000000).fit(X, y)
reg3

# Lasso Regression | L1 norm aka sum(abs(Beta)). Regularizes the loss with lambda*sum(abs(Beta))
reg4 = lm.Lasso(alpha=1.0).fit(X, y)
reg4

In [ ]:
print('y_hat_X')
reg.score(X, y)
reg.intercept_

print('y_hat_X2')
reg2.score(X2, y)
reg2.intercept_

# ridge with lambda = 1.0
print('y_hat_ridge')
reg3.score(X, y)
reg3.intercept_

# lasso with lambda = 1.0
print('y_hat_lasso')
reg4.score(X, y)
reg4.intercept_

In [ ]:
# shows that as lambda increases, the R^2 value decreases
# Also, the regular linear model had better performance, with R^2 of 0.9454
print('Running Ridge for lambda in [0.00, 0.50]')
for i in np.arange(0.00, 0.50, 0.05):
    a = round(i, 3)
    reg6 = lm.Ridge(alpha=a).fit(X, y)
    print(a)
    print(reg6.score(X, y))
    print('\n')

# Running a Lasso returns a max of about 0.9454198...
print('Running Lasso for lambda in [0.050, 0.50]')
for i in np.arange(0.050, 0.50, 0.05):
    a = round(i, 3)
    reg7 = lm.Lasso(alpha=a).fit(X, y)
    print(a)
    print(reg7.score(X, y))
    print('\n')

From the tests above, the $R^2$ to beat is 0.9454. I will continue first with testing feature selection methods. If I can achieve a sufficient improvement in $R^2$ with a subset of features, I will proceed to advanced regression techniques with that only those features. Otherwise, I will continue to performing advanced regressions with the full feature set.

In [ ]:
# Elasticnet: uses both the Ridge and Lasso penalties. Drawback of Lasso is that if there are a number of correlated 
# variables, it will only select one, and deselect for the rest. 

\begin{equation*}
{\hat {\beta }}\equiv {\underset {\beta }{\operatorname {argmin} }}(\|y-X\beta \|^{2}+\lambda _{2}\|\beta \|^2_2+\lambda _{1}\|\beta \|_{1})
\end{equation*}

scikit-learn's elasticnet implementation allows you to attribute weights (which sum to 1) to each of the penalties. Setting $l1_{ratio} = 1 $ will apply the Lasso, or $L1$ norm, penalty and setting $l1_{ratio} = 0$ will apply the Ridge, or $L2$ norm penalty. When $0 < l1_{ratio} < 1$ will apply some fraction of both penalties. The following is the objective function the implementation aims to minimize

\begin{equation*}
{\frac{1}{2*n_{samples}} * ||y - X\beta||^2_2
+ \alpha * l1_{ratio} * ||\beta||_1
+ 0.5 * \alpha * (1 - l1_{ratio}) * ||\beta||^2_2}
\end{equation*}

In [ ]:
y.head()

In [ ]:
elas_net = lm.ElasticNet(alpha=0.00001, 
                         random_state=0, 
                         l1_ratio=0.01, 
                         max_iter=100000, 
                         tol=0.001)\
                    .fit(X, y)
elas_net.score(X, y)



In [ ]:
elas_net.dual_gap_
elas_net.tol